In [67]:
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk

import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

In [1]:
# 경로 변수 (고정)


trainfile = '../_data/train_03_997.csv'
old_trainfile = '../_data/train.csv'
kaggle_trainfile = '../_data/train_k.csv'

testfile = '../_data/test_clean.csv'
old_testfile = '../_data/test.csv'

train_171016 = '../_data/171016_dow_hr_pas_L2_dur_only.csv'
train_osrm = '../_data/train_with_osrm.csv'
train_osrm_weather = '../_data/train_with_osrm_and_weather.csv'
train_osrm_weather_hourly = '../_data/train_with_osrm_and_weather_hourly.csv'
train_dropped_nan_fixed = '../_data/train__dropped__no_disaster_nan_fixed.csv'
train_added_airps = '../_data/train__dropped__lnglat_saved__airports__no_disaster.csv'
train_added_airps_cleaned = '../_data/train__dropped__cleaned__airports__no_disaster.csv'
train_added_airps_disas_cleaned = '../_data/train__dropped__cleaned__eval__airports__disaster.csv'
train_added_weekhr = '../_data/train__dropped__cleaned__eval__a_d_weekhour.csv'
train_added_manh = '../_data/train__dropped__air_bliz_weekhr_manh.csv'
train_all_vars_rough = '../_data/train__all_vars__rough.csv'

train_701778_all_vars = '../_data/train__701778__all_vars.csv'
train_700806_all_vars_cooks_dropped = '../_data/train_700806_dropped_cooks.csv'
train_700789_all_vars_cooks_dropped_circle_filter = '../_data/train_700789_dropped_cooks__circle_filter.csv'
train_700789_all_vars_cdcf_kmeans = '../_data/train_kmeans_30_added.csv'
train_700419_all_vars_cdcf150kmh_km60 = '../_data/train_kmeans_60_added_cooksd_circle_150kmh.csv'

train_700419_all_vars_4d_clust120 = '../_data/train_kmeans_4D_120.csv'
train_700419_all_vars_4d_clust120_speed = '../_data/train_kmeans_4D_120_clustspeed.csv'
train_700419_all_vars_4d_clust120_speed_glob = '../_data/train_kmeans_4D_120_clustspeed_globalized.csv'

train_700419 = '../_data/train_700419_recent.csv'
train_700419 = '../_data/train_700419_recent_2047.csv'
train_697355 = '../_data/train_697355_rsq_0722.csv'
train_697349 = '../_data/train_697349_rsq_0723.csv'
train_697284 = '../_data/train_697284_rsq_0725.csv'
train_696688 = '../_data/train_696688_rsq_0731.csv'

test_recent = '../_data/test_recent_2048.csv'

trains = [train_700419, train_697355, train_697349, train_697284, train_696688]

train_recent = trains[4]



In [3]:
pd.options.display.max_columns = 40

```
attempt k-means
```

In [9]:
# train recent
df_train = pd.read_csv(train_recent)
df_train.head()

,didnt_move,is_hol,blizz,m_d,dow,hr,p_clust,d_clust,clust_spd,p_JFK,...,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,0.0,False,0.0,430,Saturday,23,19,59,22.972353,False,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454.0
1,0.0,False,0.0,430,Saturday,23,11,0,28.539090,False,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409.0
2,0.0,False,0.0,430,Saturday,23,38,63,25.137761,False,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081.0
3,0.0,False,0.0,430,Saturday,23,35,55,11.904820,False,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800.0
4,0.0,False,0.0,430,Saturday,23,14,79,13.197395,False,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151.0


In [15]:
# train recent
df_train_696688 = pd.read_csv(train_696688)
df_train_696688.head()

,is_hol,blizz,m_d,dow,hr,wkhr,p_clust_60,d_clust_60,trip_clust_120,new_clust_spd,...,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,False,0,430,Saturday,23,143,38,11,17,22.351081,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454
1,False,0,430,Saturday,23,143,11,20,14,26.168973,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409
2,False,0,430,Saturday,23,143,9,2,14,24.123125,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081
3,False,0,430,Saturday,23,143,5,34,81,15.370883,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800
4,False,0,430,Saturday,23,143,9,50,25,13.602766,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151


In [6]:
# test recent
df_test = pd.read_csv(test_recent)
df_test.head()

,didnt_move,is_hol,blizz,m_d,dow,hr,p_clust,d_clust,clust_spd,p_JFK,...,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,0.0,False,0.0,630,Thursday,23,58,45,17.343755,False,...,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,NaN
1,0.0,False,0.0,630,Thursday,23,22,6,19.795471,False,...,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,NaN
2,0.0,False,0.0,630,Thursday,23,36,6,20.224880,False,...,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,NaN
3,0.0,False,0.0,630,Thursday,23,3,31,40.972632,False,...,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,NaN
4,0.0,False,0.0,630,Thursday,23,53,56,15.825632,False,...,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,NaN


In [36]:
fmla = 'np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + + C(dow):scale(I(hr**4)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp)'

In [16]:
fmla696688 = 'np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp)'

In [37]:
fmla

'np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + + C(dow):scale(I(hr**4)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp)'

In [17]:
mod696688 = sm.OLS.from_formula(fmla696688, data=df_train_696688)
print('m')

m


In [19]:
res696688 = mod696688.fit()

In [12]:
mod = sm.OLS.from_formula(fmla, data=df_train)
print('m')

m


In [13]:
res = mod.fit()

In [14]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     np.log(I(dur + 1))   R-squared:                       0.703
Model:                            OLS   Adj. R-squared:                  0.703
Method:                 Least Squares   F-statistic:                 2.365e+04
Date:                Wed, 25 Oct 2017   Prob (F-statistic):               0.00
Time:                        08:55:29   Log-Likelihood:            -3.7709e+05
No. Observations:              700419   AIC:                         7.543e+05
Df Residuals:                  700348   BIC:                         7.551e+05
Df Model:                          70                                         
Covariance Type:            nonrobust                                         
==========================================================================================================================
                                                             coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Intercept                                                  6.4768      0.001   5139.618      0.000       6.474       6.479
C(didnt_move)[T.1.0]                                     -79.0260     24.717     -3.197      0.001    -127.470     -30.582
C(didnt_move)[0.0]:C(is_hol)[T.True]                      -0.1241      0.004    -34.667      0.000      -0.131      -0.117
C(didnt_move)[1.0]:C(is_hol)[T.True]                      -0.1322      0.054     -2.451      0.014      -0.238      -0.026
C(didnt_move)[0.0]:C(dow)[T.Monday]                       -0.0928      0.002    -48.559      0.000      -0.097      -0.089
C(didnt_move)[1.0]:C(dow)[T.Monday]                       -0.2533      0.030     -8.459      0.000      -0.312      -0.195
C(didnt_move)[0.0]:C(dow)[T.Saturday]                     -0.1024      0.002    -57.254      0.000      -0.106      -0.099
C(didnt_move)[1.0]:C(dow)[T.Saturday]                      0.0005      0.028      0.018      0.985      -0.054       0.055
C(didnt_move)[0.0]:C(dow)[T.Sunday]                       -0.2035      0.002   -107.145      0.000      -0.207      -0.200
C(didnt_move)[1.0]:C(dow)[T.Sunday]                       -0.2496      0.029     -8.538      0.000      -0.307      -0.192
C(didnt_move)[0.0]:C(dow)[T.Thursday]                      0.0081      0.002      4.485      0.000       0.005       0.012
C(didnt_move)[1.0]:C(dow)[T.Thursday]                     -0.1621      0.028     -5.729      0.000      -0.218      -0.107
C(didnt_move)[0.0]:C(dow)[T.Tuesday]                      -0.0294      0.002    -15.768      0.000      -0.033      -0.026
C(didnt_move)[1.0]:C(dow)[T.Tuesday]                       0.0441      0.030      1.477      0.140      -0.014       0.103
C(didnt_move)[0.0]:C(dow)[T.Wednesday]                    -0.0123      0.002     -6.692      0.000      -0.016      -0.009
C(didnt_move)[1.0]:C(dow)[T.Wednesday]                    -0.1059      0.029     -3.685      0.000      -0.162      -0.050
C(didnt_move)[0.0]:C(p_LGA)[T.True]                        0.0332      0.003      9.649      0.000       0.026       0.040
C(didnt_move)[1.0]:C(p_LGA)[T.True]                        0.0603      0.027      2.225      0.026       0.007       0.113
C(didnt_move)[0.0]:C(d_JFK)[T.True]                        0.0925      0.007     13.225      0.000       0.079       0.106
C(didnt_move)[1.0]:C(d_JFK)[T.True]                       -1.7979      0.052    -34.823      0.000      -1.899      -1.697
C(didnt_move)[0.0]:C(d_LGA)[T.True]                       -0.0827      0.005    -16.181      0.000      -0.093      -0.073
C(didnt_move)[1.0]:C(d_LGA)[T.True]                        0.0603      0.027      2.225      0.026       0.007       0.113
C(didnt_

In [40]:
def get_date_rowrange(df, d1, d2):
    top = df[(df['m_d'] >= d1)
                   &(df['m_d'] < d2)].iloc[0, :].name
    bot = df[(df['m_d'] >= d1)
                   &(df['m_d'] < d2)].iloc[-1, :].name
    return top, bot

def get_month_rowranges(df):
    month_ranges = dict()

    months = [101, 201, 301, 401, 501, 601]

    for idx, month in enumerate(months):
        try:
            month_ranges[idx+1] = get_date_rowrange(df, month, month+100)
        except:
            continue
    return month_ranges

print(get_date_rowrange(df_train, 115, 211))
get_month_rowranges(df_train)


(574696, 560225)


{1: (687677, 560225), 2: (389557, 506738), 3: (279855, 277068)}

In [52]:
df_train.shape

(200000, 31)

In [50]:
df_train[(df_train['m_d'] >= 401)
                   &(df_train['m_d'] < 501)]

,didnt_move,is_hol,blizz,m_d,dow,hr,p_clust,d_clust,clust_spd,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur


In [48]:
get_date_rowrange(df_train, 401, 501)[1] - get_date_rowrange(df_train, 101, 401)[0] + 1

IndexError: single positional indexer is out-of-bounds

In [ ]:
    test_indices = random.sample(range(test_start, test_end+1),\
                                  k=test_size)

In [44]:
import random

def train_test_date_samples(X, y, train_size=None,\
                            test_size=None,\
                            train_range=(101, 401),
                            test_range=(401, 501)):
    train_start, train_end = get_date_rowrange(X, *train_range)
    test_start, test_end = get_date_rowrange(X, *test_range)

    train_cnt = train_end - train_start + 1
    test_cnt = test_end - test_start + 1
    
    if not train_size or train_size > train_cnt:
        train_size = train_cnt
    if not test_size or test_size > test_cnt:
        test_size = test_cnt
    
    train_indices = random.sample(range(train_start, train_end+1),\
                                  k=train_size)
    test_indices = random.sample(range(test_start, test_end+1),\
                                  k=test_size)
    
    print("test / (train + test) = {}".format(test_size / (train_size + test_size)))
        
    return X.loc[train_indices, :], X.loc[test_indices, :],\
           y.loc[train_indices], y.loc[test_indices]

    

X, y = df_train.iloc[:, :-1], df_train.iloc[:, -1]
    
a, b, c, d = train_test_date_samples(X, y)


IndexError: single positional indexer is out-of-bounds

In [64]:
df_train = pd.read_csv('../_data/train_700419_recent_2047.csv')
df_train.shape

(700419, 31)

In [68]:
df_trainz = df_train.copy()

In [69]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error

from datetime import datetime

t00 = datetime.now()
print(t00)

fmlas = [fmla]
df_trains = [df_trainz]

for idx in range(len(df_trains)):
    t0 = datetime.now()
    print(t0)
    
    fmla = fmlas[idx]
    df = df_trains[idx]
    scores = np.zeros(1)
    
    print("Train has {} rows".format(df.shape[0]))
    print("Test has {} rows".format(df.shape[0]))
    print("formula = {}\n".format(fmla))
    
    for i in range(2):
        X_train, X_test, y_train, y_test = train_test_date_samples(df.iloc[:, :-1].copy(), df.iloc[:, -1].copy(),\
                                                                    train_size = 200000,\
                                                                    test_size = 50000)
        df_train__ = pd.concat([X_train, y_train], axis=1)

        print("trial {}".format(i+1))
        model_train = sm.OLS.from_formula(fmla, data=df_train__)
        res_train = model_train.fit()

        y_pred = res_train.predict(X_test)
        
        y_pred = np.exp(y_pred)

        try:
            scores[i] = mean_squared_log_error(y_test, y_pred)
            print(pd.concat([y_test, y_pred], axis=1).head(5))
        except:
            print("except")
            print(df.shape)
            print(y_test[y_test<=0].shape)
            print(y_pred[y_pred<=0].shape)
            break

    scores = np.sqrt(scores)
    print(scores)
    
    t1 = datetime.now()
    print(t1)
    print(t1 - t0)
    print()
    print()

    
t2 = datetime.now()
print(t2)
print(t2 - t00)

2017-10-24 21:11:15.007711
2017-10-24 21:11:15.008688
Train has 700419 rows
Test has 700419 rows
formula = np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp) + C(didnt_move)

test / (train + test) = 0.2
trial 1
           dur            0
88933    787.0   890.580657
103732   657.0   214.226008
141881  2013.0  1348.623338
92113   1305.0   718.909701
52231    619.0   153.844245
test / (train + test) = 0.2
trial 2
except
(200000, 31)
(0,)
(0,)
[ 0.42489003]
2017-10-24 21:11:41.296093
0:00:26.287405


2017-10-24 21:11:41.296093
0:00:26.288382


In [104]:
pd.options.display.max_columns = 50

In [64]:
df_train = pd.read_csv(train_697284)
df_train.head()

,didnt_move,is_hol,blizz,m_d,dow,hr,wkhr,p_clust_60,d_clust_60,trip_clust_120,...,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,0.0,False,0,430,Saturday,23,143,38,11,17,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454
1,0.0,False,0,430,Saturday,23,143,11,20,14,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409
2,0.0,False,0,430,Saturday,23,143,9,2,14,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081
3,0.0,False,0,430,Saturday,23,143,5,34,81,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800
4,0.0,False,0,430,Saturday,23,143,9,50,25,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151


In [105]:
df_train.head()

,didnt_move,is_hol,blizz,m_d,dow,hr,wkhr,p_clust_60,d_clust_60,trip_clust_120,new_clust_spd,clust_spd,clust_spd120,L2,pas_cnt,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,avg_spd,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,0.0,False,0,430,Saturday,23,143,38,11,17,22.351081,22.351081,22.452301,0.069424,1,False,False,False,False,-73.987793,40.724792,-73.975616,40.656445,24.786272,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454
1,0.0,False,0,430,Saturday,23,143,11,20,14,26.168973,26.168973,20.823821,0.057791,1,False,False,False,False,-73.957596,40.717770,-73.951424,40.775230,26.462172,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409
2,0.0,False,0,430,Saturday,23,143,9,2,14,24.123125,24.123125,20.823821,0.066698,2,False,False,False,False,-74.000954,40.742031,-73.947708,40.782200,30.890731,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081
3,0.0,False,0,430,Saturday,23,143,5,34,81,15.370883,15.370883,13.819989,0.018223,1,False,False,False,False,-73.985733,40.738258,-73.993179,40.754890,12.271050,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800
4,0.0,False,0,430,Saturday,23,143,9,50,25,13.602766,13.602766,13.197215,0.027198,1,False,False,False,False,-74.006615,40.740650,-73.985619,40.723362,10.808758,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151


In [68]:
df_train.shape

(697284, 38)

In [65]:
df_train['clust_spd'] = df_train['new_clust_spd']

In [13]:
fmla = 'np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp)'

In [94]:
fmla = 'np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(dow):scale(I(hr**4)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp)'

In [95]:
fmla

'np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(dow):scale(I(hr**4)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp)'

In [97]:
mod = sm.OLS.from_formula(fmla, data=df_train)
res = mod.fit()

In [99]:
df_train2 = pd.read_csv('../_data/train_697284_FINAL.csv')
df_train2.head()

,is_hol,m_d,dow,hr,clust_spd,p_JFK,p_LGA,d_JFK,d_LGA,avg_spd,total_dist,total_time,day_snowdepth,hr_temp,dur
0,False,430,Saturday,23,22.351081,False,False,False,False,24.786272,10010.9,861.2,0.0,10.6,1454
1,False,430,Saturday,23,26.168973,False,False,False,False,26.462172,10357.0,786.3,0.0,10.6,1409
2,False,430,Saturday,23,24.123125,False,False,False,False,30.890731,9275.8,703.9,0.0,10.6,1081
3,False,430,Saturday,23,15.370883,False,False,False,False,12.271050,2726.9,298.0,0.0,10.6,800
4,False,430,Saturday,23,13.602766,False,False,False,False,10.808758,3455.8,458.2,0.0,10.6,1151


In [101]:
df_train2.head()

,is_hol,m_d,dow,hr,clust_spd,p_JFK,p_LGA,d_JFK,d_LGA,avg_spd,total_dist,total_time,day_snowdepth,hr_temp,dur
0,False,430,Saturday,23,22.351081,False,False,False,False,24.786272,10010.9,861.2,0.0,10.6,1454
1,False,430,Saturday,23,26.168973,False,False,False,False,26.462172,10357.0,786.3,0.0,10.6,1409
2,False,430,Saturday,23,24.123125,False,False,False,False,30.890731,9275.8,703.9,0.0,10.6,1081
3,False,430,Saturday,23,15.370883,False,False,False,False,12.271050,2726.9,298.0,0.0,10.6,800
4,False,430,Saturday,23,13.602766,False,False,False,False,10.808758,3455.8,458.2,0.0,10.6,1151


In [102]:
mod = sm.OLS.from_formula(fmla, data=df_train2)
res = mod.fit()

In [98]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     np.log(I(dur + 1))   R-squared:                       0.730
Model:                            OLS   Adj. R-squared:                  0.730
Method:                 Least Squares   F-statistic:                 4.493e+04
Date:                Wed, 25 Oct 2017   Prob (F-statistic):               0.00
Time:                        10:31:03   Log-Likelihood:            -3.3037e+05
No. Observations:              697284   AIC:                         6.608e+05
Df Residuals:                  697241   BIC:                         6.613e+05
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               6.4978      0.001   5498.474      0.000       6.495       6.500
C(is_hol)[T.True]                      -0.1268      0.003    -37.750      0.000      -0.133      -0.120
C(dow)[T.Monday]                       -0.0904      0.002    -50.411      0.000      -0.094      -0.087
C(dow)[T.Saturday]                     -0.1049      0.002    -62.547      0.000      -0.108      -0.102
C(dow)[T.Sunday]                       -0.2047      0.002   -114.952      0.000      -0.208      -0.201
C(dow)[T.Thursday]                      0.0079      0.002      4.647      0.000       0.005       0.011
C(dow)[T.Tuesday]                      -0.0272      0.002    -15.551      0.000      -0.031      -0.024
C(dow)[T.Wednesday]                    -0.0104      0.002     -6.055      0.000      -0.014      -0.007
C(p_LGA)[T.True]                        0.0098      0.003      3.042      0.002       0.003       0.016
C(d_JFK)[T.True]                        0.0837      0.007     12.762      0.000       0.071       0.097
C(d_LGA)[T.True]                       -0.0909      0.005    -18.949      0.000      -0.100      -0.082
C(dow)[Friday]:scale(hr)               -0.6063      0.021    -28.407      0.000      -0.648      -0.564
C(dow)[Monday]:scale(hr)               -0.3616      0.027    -13.415      0.000      -0.414      -0.309
C(dow)[Saturday]:scale(hr)             -0.7839      0.020    -38.274      0.000      -0.824      -0.744
C(dow)[Sunday]:scale(hr)               -0.8371      0.021    -39.360      0.000      -0.879      -0.795
C(dow)[Thursday]:scale(hr)             -0.5758      0.024    -24.060      0.000      -0.623      -0.529
C(dow)[Tuesday]:scale(hr)              -0.5423      0.026    -20.601      0.000      -0.594      -0.491
C(dow)[Wednesday]:scale(hr)            -0.6720      0.025    -26.745      0.000      -0.721      -0.623
C(dow)[Friday]:scale(I(hr ** 2))        4.5446      0.097     46.675      0.000       4.354       4.735
C(dow)[Monday]:scale(I(hr ** 2))        3.9188      0.120     32.699      0.000       3.684       4.154
C(dow)[Saturday]:scale(I(hr ** 2))      3.3295      0.093     35.854      0.000       3.147       3.511
C(dow)[Sunday]:scale(I(hr ** 2))        3.1906      0.098     32.534      0.000       2.998       3.383
C(dow)[Thursday]:scale(I(hr ** 2))      5.0811      0.107     47.315      0.000       4.871       5.292
C(dow)[Tuesday]:scale(I(hr ** 2))       5.1528      0.116     44.323      0.000       4.925       5.381
C(dow)[Wednesday]:scale(I(hr ** 2))     5.6022      0.112     50.038      0.000       5.383       5.822
C(dow)[Friday]:scale(I(hr ** 3))       -7.1981      0.143    -50.174      0.000      -7.479      -6.917
C(dow)[Monday]:scale(I(hr ** 3))       -6.6022      0.176    -37.54

In [98]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     np.log(I(dur + 1))   R-squared:                       0.730
Model:                            OLS   Adj. R-squared:                  0.730
Method:                 Least Squares   F-statistic:                 4.493e+04
Date:                Wed, 25 Oct 2017   Prob (F-statistic):               0.00
Time:                        10:31:03   Log-Likelihood:            -3.3037e+05
No. Observations:              697284   AIC:                         6.608e+05
Df Residuals:                  697241   BIC:                         6.613e+05
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               6.4978      0.001   5498.474      0.000       6.495       6.500
C(is_hol)[T.True]                      -0.1268      0.003    -37.750      0.000      -0.133      -0.120
C(dow)[T.Monday]                       -0.0904      0.002    -50.411      0.000      -0.094      -0.087
C(dow)[T.Saturday]                     -0.1049      0.002    -62.547      0.000      -0.108      -0.102
C(dow)[T.Sunday]                       -0.2047      0.002   -114.952      0.000      -0.208      -0.201
C(dow)[T.Thursday]                      0.0079      0.002      4.647      0.000       0.005       0.011
C(dow)[T.Tuesday]                      -0.0272      0.002    -15.551      0.000      -0.031      -0.024
C(dow)[T.Wednesday]                    -0.0104      0.002     -6.055      0.000      -0.014      -0.007
C(p_LGA)[T.True]                        0.0098      0.003      3.042      0.002       0.003       0.016
C(d_JFK)[T.True]                        0.0837      0.007     12.762      0.000       0.071       0.097
C(d_LGA)[T.True]                       -0.0909      0.005    -18.949      0.000      -0.100      -0.082
C(dow)[Friday]:scale(hr)               -0.6063      0.021    -28.407      0.000      -0.648      -0.564
C(dow)[Monday]:scale(hr)               -0.3616      0.027    -13.415      0.000      -0.414      -0.309
C(dow)[Saturday]:scale(hr)             -0.7839      0.020    -38.274      0.000      -0.824      -0.744
C(dow)[Sunday]:scale(hr)               -0.8371      0.021    -39.360      0.000      -0.879      -0.795
C(dow)[Thursday]:scale(hr)             -0.5758      0.024    -24.060      0.000      -0.623      -0.529
C(dow)[Tuesday]:scale(hr)              -0.5423      0.026    -20.601      0.000      -0.594      -0.491
C(dow)[Wednesday]:scale(hr)            -0.6720      0.025    -26.745      0.000      -0.721      -0.623
C(dow)[Friday]:scale(I(hr ** 2))        4.5446      0.097     46.675      0.000       4.354       4.735
C(dow)[Monday]:scale(I(hr ** 2))        3.9188      0.120     32.699      0.000       3.684       4.154
C(dow)[Saturday]:scale(I(hr ** 2))      3.3295      0.093     35.854      0.000       3.147       3.511
C(dow)[Sunday]:scale(I(hr ** 2))        3.1906      0.098     32.534      0.000       2.998       3.383
C(dow)[Thursday]:scale(I(hr ** 2))      5.0811      0.107     47.315      0.000       4.871       5.292
C(dow)[Tuesday]:scale(I(hr ** 2))       5.1528      0.116     44.323      0.000       4.925       5.381
C(dow)[Wednesday]:scale(I(hr ** 2))     5.6022      0.112     50.038      0.000       5.383       5.822
C(dow)[Friday]:scale(I(hr ** 3))       -7.1981      0.143    -50.174      0.000      -7.479      -6.917
C(dow)[Monday]:scale(I(hr ** 3))       -6.6022      0.176    -37.54

In [71]:
del df_test['dur']

In [70]:
df_test = pd.read_csv(test_recent)
df_test.head()

,didnt_move,is_hol,blizz,m_d,dow,hr,p_clust,d_clust,clust_spd,p_JFK,...,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,0.0,False,0.0,630,Thursday,23,58,45,17.343755,False,...,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,NaN
1,0.0,False,0.0,630,Thursday,23,22,6,19.795471,False,...,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,NaN
2,0.0,False,0.0,630,Thursday,23,36,6,20.224880,False,...,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,NaN
3,0.0,False,0.0,630,Thursday,23,3,31,40.972632,False,...,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,NaN
4,0.0,False,0.0,630,Thursday,23,53,56,15.825632,False,...,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,NaN


In [91]:
df_train.shape

(697284, 38)

In [92]:
fmla

'np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + + C(dow):scale(I(hr**4)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp)'

In [72]:
df_test.head()

,didnt_move,is_hol,blizz,m_d,dow,hr,p_clust,d_clust,clust_spd,p_JFK,...,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,0.0,False,0.0,630,Thursday,23,58,45,17.343755,False,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
1,0.0,False,0.0,630,Thursday,23,22,6,19.795471,False,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
2,0.0,False,0.0,630,Thursday,23,36,6,20.224880,False,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
3,0.0,False,0.0,630,Thursday,23,3,31,40.972632,False,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0
4,0.0,False,0.0,630,Thursday,23,53,56,15.825632,False,...,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0


In [73]:
test_pred_log = res.predict(df_test)

In [74]:
test_pred = np.exp(test_pred_log)

In [75]:
test_pred.head()

0    1129.244008
1     450.971181
2    2093.444323
3    1510.510948
4     834.830720
dtype: float64

In [76]:
df_test.loc[:, 'dur'] = test_pred

In [77]:
avg_zero = 503.51274886482713

In [78]:
df_test.loc[df_test['total_dist'] == 0.0, 'dur'] = avg_zero

In [79]:
df_test.loc[df_test['didnt_move'] == 1, :]

,didnt_move,is_hol,blizz,m_d,dow,hr,p_clust,d_clust,clust_spd,p_JFK,...,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
394,1.0,False,0.0,630,Thursday,22,47,19,16.419546,False,...,0.00,0.0,0.0,25.0,0.58,16.1,5.6,0.0,0.0,503.512749
490,1.0,False,0.0,630,Thursday,22,37,44,20.224880,False,...,0.00,0.0,0.0,25.0,0.58,16.1,5.6,0.0,0.0,503.512749
763,1.0,False,0.0,630,Thursday,21,14,12,18.904541,False,...,0.00,0.0,0.0,25.6,0.54,15.6,9.3,0.0,0.0,503.512749
931,1.0,False,0.0,630,Thursday,21,72,15,15.655126,False,...,0.00,0.0,0.0,25.6,0.54,15.6,9.3,0.0,0.0,503.512749
1518,1.0,False,0.0,630,Thursday,19,72,15,15.655126,False,...,0.00,0.0,0.0,27.2,0.44,13.9,7.4,0.0,0.0,503.512749
1917,1.0,False,0.0,630,Thursday,18,69,4,25.178375,False,...,0.00,0.0,0.0,28.9,0.37,12.8,0.0,0.0,0.0,503.512749
2231,1.0,False,0.0,630,Thursday,16,62,74,13.399223,False,...,0.00,0.0,0.0,28.3,0.36,11.7,0.0,0.0,0.0,503.512749
2442,1.0,False,0.0,630,Thursday,15,2,4,24.576087,False,...,0.00,0.0,0.0,28.9,0.38,13.3,0.0,0.0,0.0,503.512749
2487,1.0,False,0.0,630,Thursday,15,24,61,14.466893,False,...,0.00,0.0,0.0,28.9,0.38,13.3,0.0,0.0,0.0,503.512749
2526,1.0,False,0.0,630,Thursday,15,0,16,19.342633,False,...,0.00,0.0,0.0,28.9,0.38,13.3,0.0,0.0,0.0,503.512749


In [80]:
test_pred = df_test.loc[:, 'dur']

In [82]:
test_pred.loc[393:394]

393    2022.299918
394     503.512749
Name: dur, dtype: float64

In [83]:
df_test.loc[:, 'dur'] = test_pred

In [24]:
avg_zero = 503.51274886482713
X_test.loc[:, 'dur'] = y_pred
X_test.loc[X_test['didnt_move'] == 1, 'dur'] = avg_zero
y_pred = X_test.loc[:, 'dur']

In [28]:
test_pred

503.51274886482713

In [84]:
df_test_orig = pd.read_csv('../_data/test.csv')
df_test_orig.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id2332349,2,6/30/16 23:59,5,-73.998291,40.722618,-73.971779,40.762470,N
1,id2013516,1,6/30/16 23:59,2,-73.955109,40.689564,-73.978203,40.685802,N
2,id3719493,2,6/30/16 23:59,1,-73.978416,40.791576,-73.973160,40.675968,N
3,id0376262,1,6/30/16 23:59,2,-73.873093,40.774097,-73.926704,40.856739,N
4,id2105228,1,6/30/16 23:58,1,-73.979416,40.755211,-74.003548,40.738224,N


In [85]:
df_test_orig.shape

(346797, 9)

In [86]:
test_pred.shape

(346797,)

In [87]:
df_final = pd.concat([df_test_orig['id'], test_pred], axis=1, names=['id', 'trip_duration'])

In [88]:
df_final.columns = ['id', 'trip_duration']

In [89]:
df_final.loc[394, :]

id               id2447155
trip_duration      503.513
Name: 394, dtype: object

In [90]:
df_final.to_csv('../_data/submission_0959.txt', index=False)

In [35]:
fmla

'np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp)'

In [34]:
df_final.isnull().sum()

id               0
trip_duration    0
dtype: int64

In [93]:
df_final_backup = df_final

replace 0 dist values

In [1]:
df_raw = pd.read_csv('../_data/train__701778__all_vars.csv')
df_raw.head()

,is_hol,blizz,m_d,dow,hr,wkhr,L2,pas_cnt,p_JFK,p_LGA,...,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,False,0,430,Saturday,23,143,0.069424,1,False,False,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454
1,False,0,430,Saturday,23,143,0.057791,1,False,False,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409
2,False,0,430,Saturday,23,143,0.066698,2,False,False,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081
3,False,0,430,Saturday,23,143,0.018223,1,False,False,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800
4,False,0,430,Saturday,23,143,0.027198,1,False,False,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151


In [18]:
avg_zero = df_raw.loc[df_raw['total_dist'] == 0, 'dur'].mean()
avg_zero

503.51274886482713

In [107]:
df_test['dur'] = test_pred

In [108]:
df_test.head()

,didnt_move,is_hol,blizz,m_d,dow,hr,p_clust,d_clust,clust_spd,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,0.0,False,0.0,630,Thursday,23,58,45,17.343755,False,False,False,False,-73.998291,40.722618,-73.971779,40.762470,5280.3,623.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,1007.790371
1,0.0,False,0.0,630,Thursday,23,22,6,19.795471,False,False,False,False,-73.955109,40.689564,-73.978203,40.685802,2573.5,219.3,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,408.912734
2,0.0,False,0.0,630,Thursday,23,36,6,20.224880,False,False,False,False,-73.978416,40.791576,-73.973160,40.675968,15048.8,1300.0,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,1850.600478
3,0.0,False,0.0,630,Thursday,23,3,31,40.972632,False,True,False,False,-73.873093,40.774097,-73.926704,40.856739,16818.3,1029.1,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,1382.675226
4,0.0,False,0.0,630,Thursday,23,53,56,15.825632,False,False,False,False,-73.979416,40.755211,-74.003548,40.738224,3536.7,434.5,85,67,0.0,0.0,0.0,24.4,0.6,16.1,0.0,0.0,0.0,748.497554


In [ ]:
'p_lng', 'p_lat', 'd_lng', 'd_lat'

In [110]:
df_test_small = df_test[['id', 'didnt_move', 'p_lng', 'p_lat', 'd_lng', 'd_lat', 'total_dist', 'dur']].copy()

In [114]:
df_test_small.loc[df_test_small['didnt_move'] == 1, 'dur'] = avg_zero

In [118]:
df_test_small.insert(0, 'id', df_test_orig['id'])
df_test_small.head()

,id,didnt_move,p_lng,p_lat,d_lng,d_lat,total_dist,dur
0,id2332349,0.0,-73.998291,40.722618,-73.971779,40.762470,5280.3,1007.790371
1,id2013516,0.0,-73.955109,40.689564,-73.978203,40.685802,2573.5,408.912734
2,id3719493,0.0,-73.978416,40.791576,-73.973160,40.675968,15048.8,1850.600478
3,id0376262,0.0,-73.873093,40.774097,-73.926704,40.856739,16818.3,1382.675226
4,id2105228,0.0,-73.979416,40.755211,-74.003548,40.738224,3536.7,748.497554


In [121]:
df_final_2 = df_test_small.loc[:, ['id', 'dur']]

In [126]:
df_final_2.columns = ['id', 'trip_duration']
df_final_2.to_csv('../_data/submission_2131.txt', index=False)

In [115]:
df_test_small.loc[df_test_small['total_dist'] == 0.0, :]

,didnt_move,p_lng,p_lat,d_lng,d_lat,total_dist,dur
394,1.0,-73.892860,40.765896,-73.892860,40.765896,0.0,503.512749
490,1.0,-74.325340,40.723003,-74.325340,40.723003,0.0,503.512749
763,1.0,-74.030060,40.749531,-74.030060,40.749531,0.0,503.512749
931,1.0,-74.040298,40.621075,-74.040298,40.621075,0.0,503.512749
1518,1.0,-74.051002,40.604469,-74.051002,40.604469,0.0,503.512749
1917,1.0,-73.794342,40.657082,-73.794342,40.657082,0.0,503.512749
2231,1.0,-74.005089,40.731529,-74.005089,40.731529,0.0,503.512749
2442,1.0,-73.776337,40.645329,-73.776337,40.645329,0.0,503.512749
2487,1.0,-73.945427,40.751575,-73.945427,40.751575,0.0,503.512749
2526,1.0,-73.964729,40.771557,-73.964729,40.771557,0.0,503.512749


uses train_test_date_sample2

In [29]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error

from datetime import datetime

t00 = datetime.now()
print(t00)

df0 = df_trains[0]
scores_all = list()

for idx in range(len(df_trains)):
    t0 = datetime.now()
    print(t0)
    
    fmla = fmlas[idx]
    df = df_trains[idx]
    scores = np.zeros(1)
    
    print("Train has {} rows".format(df.shape[0]))
    print("Test has {} rows".format(df0.shape[0]))
    print("formula = {}\n".format(fmla))
    
    for i in range(1):
        X_train, X_test, y_train, y_test = train_test_date_samples2(df.iloc[:, :-1], df.iloc[:, -1],\
                                                                    df0.iloc[:, :-1], df0.iloc[:, -1],\
                                                                    train_size = 200000,\
                                                                    test_size = 50000)
        df_train = pd.concat([X_train, y_train], axis=1)

        print("trial {}".format(i+1))
        model_train = sm.OLS.from_formula(fmla, data=df_train)
        res_train = model_train.fit()

        y_pred = res_train.predict(X_test)
        
        y_pred = np.exp(y_pred)

        try:
            scores[i] = mean_squared_log_error(y_test, y_pred)
            print(pd.concat([y_test, y_pred], axis=1).head(5))
        except:
            print("except")
            print(y_test[y_test<=0].shape)
            print(y_pred[y_pred<=0].shape)
            break

    scores = np.sqrt(scores)
    print(scores)
    scores_all.append(scores)
    
    t1 = datetime.now()
    print(t1)
    print(t1 - t0)
    print()
    print()

    
t2 = datetime.now()
print(t2)
print(t2 - t00)

2017-10-24 20:57:08.980862
2017-10-24 20:57:08.981864
Train has 700419 rows


NameError: name 'df0' is not defined

In [151]:
scores_all

array([ 0.4229259,  0.4229259,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ])

In [36]:
res_list = list()

for idx in len(df_trains):
    print(idx, end=' ')
    mod = sm.OLS.from_formula(df_res.iloc[idx, 0], data=df_trains[idx])
    res_list.append(mod)

'np.log(I(dur+1)) ~ C(is_hol) + C(dow) + C(dow):scale(hr) + C(dow):scale(I(hr**2)) + C(dow):scale(I(hr**3)) + C(p_LGA) + C(d_JFK) + C(d_LGA) + scale(new_clust_spd) + scale(np.log(I(total_time+1))) + scale(day_snowdepth) + scale(hr_temp) + C(didnt_move)'

In [181]:
df_testz.to_csv('../_data/test_recent_2048.csv', index=False)

In [35]:
import random

def train_test_date_samples2(X, y, X0, y0, train_size=None,\
                            test_size=None,\
                            train_range=(101, 401),
                            test_range=(401, 501)):
    train_start, train_end = get_date_rowrange(X, *train_range)
    test_start, test_end = get_date_rowrange(X, *test_range)

    train_cnt = train_end - train_start + 1
    test_cnt = test_end - test_start + 1
    
    if not train_size or train_size > train_cnt:
        train_size = train_cnt
    if not test_size or test_size > test_cnt:
        test_size = test_cnt
    
    train_indices = random.sample(range(train_start, train_end+1),\
                                  k=train_size)
    test_indices = random.sample(range(test_start, test_end+1),\
                                  k=test_size)
    
    print("test / (train + test) = {}".format(test_size / (train_size + test_size)))
        
    return X.loc[train_indices, :], X0.loc[test_indices, :],\
           y.loc[train_indices], y0.loc[test_indices]

    

X, y = df_train.iloc[:, :-1], df_train.iloc[:, -1]
    
a, b, c, d = train_test_date_samples2(X, y, X, y)


IndexError: single positional indexer is out-of-bounds